# Lecture 07 — Market Research and Analysis

**テーマ:** クラスターを説明するための外部変数（genderなど）の扱いと、定量変数による記述。

**学習目標**
1. 2つのカテゴリー変数間の独立性 (Chi-square test)
2. Goodman–Kruskal’s Lambda（予測力の向上度）
3. Quantitative variables によるクラスターの特徴づけ（平均値比較）


## 0) データ準備

使用データ: `c.unico_2`  
変数: `id`, `cluster`, `sex`, `d10_1`〜`d10_8`


In [ ]:
* path for students;
libname c 'C:\Camillo_clabe';
* path for Furio;
libname c '\\MaterialeDocente\Camillo_clabe';

/* 使用データ: c.unico_2
   変数: id, cluster, sex, d10_1–d10_8 */


## 1) Categorical variables の関係 — Chi-square test

目的: **cluster** と **sex** の間に統計的な関係（独立性）があるかを確認する。


In [ ]:
proc freq data=c.unico_2;
  tables cluster*sex / expected chisq all;
run;

### 読み方
- `expected` は独立である場合の期待度数
- `chisq` は Chi-square 統計量と p-value
- 授業の実行では p-value ≈ 0.47 で独立とみなされた


### クラスター3を除いた 2×2 テーブルの確認

In [ ]:
proc freq data=c.unico_2;
  where cluster ne 3;
  tables cluster*sex / expected chisq;
run;

## 2) Goodman–Kruskal’s Lambda

目的: 性別 (sex) を知ることで **cluster** の予測精度がどのくらい向上するかを測る。


In [ ]:
proc freq data=c.unico_2;
  tables cluster*sex / measures;   /* Lambda を含む関連指標を出力 */
run;

### 読み方
- 出力の「Asymmetric Measures」に **Lambda (Rows Dependent on Columns)** がある
- 授業では Lambda ≈ 0.02（約2%の予測力向上）で、性別は弱い予測因子
- 結論: 設計において gender を考慮しなくてよい


## 3) Quantitative variables によるクラスター記述

目的: 意見項目 `d10_1`〜`d10_8` について、クラスターごとの平均を比較して特徴を把握する。


### 3a) 全体平均の算出

In [ ]:
proc means data=c.unico_2 mean stddev min max;
  var d10_1-d10_8;
  title "Overall means for opinion variables";
run;

### 3b) クラスター別の平均値

In [ ]:
proc means data=c.unico_2 mean stddev min max;
  class cluster;
  var d10_1-d10_8;
  title "Cluster means for opinion variables";
run;

### 読み方
- クラスター平均が全体平均より大きいと over-characterized
- クラスター平均が全体平均より小さいと under-characterized


### 3c) 平均差 (cluster mean − overall mean) の簡易表

In [ ]:
/* Overall means を一時データに保存 */
proc means data=c.unico_2 noprint;
  var d10_1-d10_8;
  output out=overall mean=om1-om8;
run;

/* macro 変数として保存 */
data _null_;
  set overall;
  call symputx('om1', om1);
  call symputx('om2', om2);
  call symputx('om3', om3);
  call symputx('om4', om4);
  call symputx('om5', om5);
  call symputx('om6', om6);
  call symputx('om7', om7);
  call symputx('om8', om8);
run;

/* cluster 別平均と全体との差を計算 */
proc means data=c.unico_2 noprint;
  class cluster;
  var d10_1-d10_8;
  output out=byclust mean=m1-m8;
run;

data deltas;
  set byclust;
  where _TYPE_ = 1; /* クラスタ行のみ */
  d10_1_delta = m1 - &om1;
  d10_2_delta = m2 - &om2;
  d10_3_delta = m3 - &om3;
  d10_4_delta = m4 - &om4;
  d10_5_delta = m5 - &om5;
  d10_6_delta = m6 - &om6;
  d10_7_delta = m7 - &om7;
  d10_8_delta = m8 - &om8;
  keep cluster d10_1_delta-d10_8_delta;
run;

proc print data=deltas label noobs;
  title "Cluster mean minus overall mean (Δ)";
run;

## まとめ

| 内容 | 方法 | 解釈 |
|------|------|------|
| Categorical vs categorical | `PROC FREQ` + `CHISQ` | p ≈ 0.47 で独立 |
| Predictive relation | `PROC FREQ` + `MEASURES` | Lambda ≈ 0.02、性別は弱い予測因子 |
| Quantitative variables | `PROC MEANS` | クラスター平均の比較で特徴づけ |

**最終結論:** Gender はクラスターに影響を与えないため、設計では意見データを主軸にする。
